Just an attempt; but a massive fail

In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np

The system:

\begin{align}
\partial_x A &= A \left[ \frac{1-A}{x(1-x)} + \frac{8 \pi x A \rho}{(1-x)^3} \right] \,,
\\
\partial_x \alpha & = \alpha \left[ \frac{A-1}{2x(1-x)} + \frac{4 \pi A S_A}{(1-x)^3} \right] \,,
\\
\partial_x \chi & = - \frac{\chi}{x(1-x)} \left[ 1 + A - \frac{8 \pi x A V(\phi)}{(1-x)} \right] + 
    \frac{A}{(1-x)^2} \left[ \frac{dV}{d\phi} - \left( \frac{\omega}{\alpha} \right)^2 \phi \right] \,,
\\
\partial_x \phi & = \frac{\chi}{(1-x)^2}
\,.
\end{align}

Boundary conditions:
\begin{align}
A(x=0) = 1 \,, \quad \partial_r \alpha(x=0) = 0 \,, \quad \phi(x=0) = \phi_0 \,, \quad \chi(x=0) = 0 \,,
\\
A(x=1) = 1 \,, \quad \partial_r \alpha(x=1) = 1 \,, \quad \phi(x=1) = 0 \,, \quad \chi(x=1) = 0 \,.
\end{align}

The potential is
$$
V(\phi) = \frac{1}{2} m^2 \phi^2 \,, \quad \frac{d V}{d \phi} = m^2 \phi \,.
$$

In [ ]:
# copied from https://github.com/benmoseley/harmonic-oscillator-pinn/blob/main/Harmonic%20oscillator%20PINN.ipynb
class FCN(torch.nn.Module):   
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = torch.nn.Tanh
        self.fcs = torch.nn.Sequential(*[
                        torch.nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        self.fch = torch.nn.Sequential(*[
                        torch.nn.Sequential(*[
                            torch.nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        self.fce = torch.nn.Linear(N_HIDDEN, N_OUTPUT)
        
    def forward(self, x):
        x = self.fcs(x)
        x = self.fch(x)
        x = self.fce(x)
        return x

In [ ]:
# copied from https://github.com/raimonluna/MachineLearningForStrongGravity/blob/main/Lecture1_Physics_Informed_Neural_Networks.ipynb
def gradients(outputs, inputs, order = 1):
    if order == 1:
        return torch.autograd.grad(outputs, inputs, grad_outputs=torch.ones_like(outputs), create_graph=True)[0]
    elif order > 1:
        return gradients(gradients(outputs, inputs, 1), inputs, order - 1)
    else:
        return outputs

In [ ]:
# random (uniform) sample points
def random_domain_points(xmax, n):
    #x = xmax*torch.rand((n,1), requires_grad=True) + epsilon*torch.ones((n,1), requires_grad=True)
    x = xmax*torch.rand((n,1), requires_grad=True)
    #if torch.is_nonzero(x)==True:
    #    print("no zero")
    #else:
    #    print("there is a zero")
    return x

In [ ]:
def domain_loss(u, x, omega, m, eps): # eps is epsilon <<1 to never have vanishing denominators 
    # torch.transpose(ux.detach(), 0, 1) remove some info and transpose, then [0] is Ax, etc
    #A = torch.transpose(u.detach(), 0, 1)[0].view(-1,1)
    #alpha = torch.transpose(u.detach(), 0, 1)[1].view(-1,1)
    #chi = torch.transpose(u.detach(), 0, 1)[2].view(-1,1)
    #phi = torch.transpose(u.detach(), 0, 1)[3].view(-1,1)
    # output same as above
    # from https://github.com/raimonluna/MachineLearningForStrongGravity/blob/main/Lecture1_Physics_Informed_Neural_Networks.ipynb
    A, alpha, chi, phi = map(lambda i:  u[:,[i]], range(4))
    # take derivatives
    Ax = gradients(A, x)
    alphax = gradients(alpha, x)
    chix = gradients(chi, x)
    phix = gradients(phi, x)

    # potential
    V = 0.5*torch.pow(m,2)*torch.pow(phi,2)
    # potential derivative wrt phi
    dVdphi = torch.pow(m,2)*phi
    # rho
    rho = 0.5*(torch.pow(chi,2)/A + torch.pow((omega/alpha),2)*torch.pow(phi,2)) + V
    # S_A
    SA = 0.5*(torch.pow(chi,2)/A + torch.pow((omega/alpha),2)*torch.pow(phi,2)) - V
    # eq_A is \p_rA - rhs[A]
    eq_A = Ax - A*( (1-A)/((x+eps)*(1-x+eps)) + 8*torch.pi*x*A*rho/torch.pow((1-x+eps),3) )
    # eq_alpha is \p_r alpha - rhs[alpha]
    eq_alpha = alphax - alpha*( (A-1)/(2*(x+eps)*(1-x+eps)) + 4*torch.pi*A*SA/torch.pow((1-x+eps),3) )
    # eq_chi  is \p_r chi - rhs[chi]
    eq_chi = - (chi/((x+eps)*(1-x+eps)))*(1 + A - 8*torch.pi*x*A*V/(1-x+eps)) + (A/torch.pow((1-x+eps),2))*(dVdphi - torch.pow((omega/alpha),2)*phi)
    # eq_phi is \p_r phi - rhs[phi]
    eq_phi = chi/torch.pow((1-x+eps),2)

    loss_dom = torch.mean(torch.pow(eq_A,2)) + torch.mean(torch.pow(eq_alpha,2)) + torch.mean(torch.pow(eq_chi,2)) + torch.mean(torch.pow(eq_phi,2))
    return loss_dom

def x0_loss(u0, x0, phi0):
    #alphax = torch.transpose(ux0.detach(), 0, 1)[1].view(-1,1)
    #A = torch.transpose(u0.detach(), 0, 1)[0].view(-1,1)
    #chi = torch.transpose(u0.detach(), 0, 1)[2].view(-1,1)
    #phi = torch.transpose(u0.detach(), 0, 1)[3].view(-1,1)
    # from https://github.com/raimonluna/MachineLearningForStrongGravity/blob/main/Lecture1_Physics_Informed_Neural_Networks.ipynb
    A, alpha, chi, phi = map(lambda i:  u0[[i]], range(4))
    # take derivatives
    alphax = gradients(alpha, x0)
    
    loss_x0 = torch.mean(torch.pow(A-1,2)) + torch.mean(torch.pow(alphax,2)) + torch.mean(torch.pow(phi-phi0,2)) + torch.mean(torch.pow(chi,2))
    return loss_x0

def xmax_loss(umax):
    #A = torch.transpose(umax.detach(), 0, 1)[0].view(-1,1)
    #alpha = torch.transpose(umax.detach(), 0, 1)[1].view(-1,1)
    #chi = torch.transpose(umax.detach(), 0, 1)[2].view(-1,1)
    #phi = torch.transpose(umax.detach(), 0, 1)[3].view(-1,1)
    # from https://github.com/raimonluna/MachineLearningForStrongGravity/blob/main/Lecture1_Physics_Informed_Neural_Networks.ipynb
    A, alpha, chi, phi = map(lambda i:  umax[[i]], range(4))
    
    loss_xmax = torch.mean(torch.pow(A-1,2)) + torch.mean(torch.pow(alpha-1,2)) + torch.mean(torch.pow(phi,2)) + torch.mean(torch.pow(chi,2))
    return loss_xmax

In [ ]:
torch.manual_seed(123)

# input 1 (x), output 4 (A, alpha, phi, chi), 32 nodes per layer, 3 hidden layers
model = FCN(1,4,32,3)

omega = torch.nn.Parameter(1*torch.ones(1, requires_grad=True))
optimizer = torch.optim.Adam(list(model.parameters())+[omega],lr=5e-4)

In [ ]:
n = 2048 # number of random sampling points

epochs = 1e6
gamma1 = 10.0
gamma2 = 10.0
# epsilon is for the random x points, not to get the value 0
epsilon = 1.e-4

# xmax
X = 1
# mass
m = 1*torch.ones(1)
# phi(x=0)
phi0 = 1*torch.ones(1)

# lists to save things
loss_list = []
omegas = []

for epoch in range(int(epochs)):
    optimizer.zero_grad() # to make the gradients zero
    # x=0
    x0 = torch.zeros(1, requires_grad=True)
    # xmax
    xmax = X*torch.ones(1, requires_grad=True)
    # time sample
    x = random_domain_points(X, n)
    u = model(x)
    #print(u)
    # Derivatives
    #ux = torch.autograd.grad(outputs=u, 
    #                         inputs=x,
    #                         create_graph=True,
    #                         grad_outputs=torch.ones_like(u)
    #                         )[0]
    #print(ux)#? i expect 4 outputs and i get 1 (var)
    # loss for the bulk of the domain
    loss_dom = domain_loss(u, x, omega, m, epsilon)
    # boundary data at x=0
    u0 = model(x0)
    #print(u0)
    #ux0 = torch.autograd.grad(outputs=u0, 
    #                          inputs=x0,
    #                          create_graph=True,
    #                          grad_outputs=torch.ones_like(u0)
    #                          )#[0]
    #print(ux0)
    loss_x0 = x0_loss(u0, x0, phi0)
    # boundary data at x=xmax
    umax = model(xmax)
    loss_xmax = xmax_loss(umax)
    # LOSS
    loss = loss_dom + gamma1*loss_x0 + gamma2*loss_xmax
    # save loss and omega values
    loss_list.append(loss.detach().numpy())
    omegas.append(omega.item())
    # print message
    print('epoch = ', epoch, '| loss = ', loss_list[-1], ' | omega = ', omegas[-1], '|',  end='\r')
    # detach() removes the "requires_grad" and numpy() makes it a numpy item to plot later
    loss.backward() # This is for computing gradients using backward propagation
    optimizer.step() # 


In [ ]:
plt.semilogy(loss_list)

In [ ]:
plt.plot(omegas)